# PyEumap - Overlay Demonstration

In this tutorial, we will use the pyeumap package to overlay all the points of a vector layer (*geopackage file*) on several raster layers (*geotiff files*), using the **SpaceOverlay** and **SpaceTimeOverlay** classes to handle with timeless and temporal layers, respectively. 

In our dataset the elevation and slope, based on digital terrain model, are timeless and the landsat composites (7 spectral bands, 4 seasons and 3 percentiles) and night light (VIIRS Night Band) layers are temporal (from 2000 to 2020).

First, let's import the necessary modules

In [1]:
import os
from pathlib import Path
import sys

import pandas as pd
import geopandas as gpd

# Add the repository root in the path
# If the pyeumap isn't instaled you should do it   
sys.path.append('../../')

from pyeumap.overlay import SpaceOverlay, SpaceTimeOverlay

## Dataset 

Our dataset refers to 1 tile, located in Sweden, extracted from a tiling system created for European Union (7,042 tiles) by [GeoHarmonizer Project](https://opendatascience.eu).

In [2]:
data_dir = '/home/leandro/Tmp/pilot_tiles/croatia_9529'

For this tile we have a **geopackage** file containing the points

In [3]:
fn_points = os.path.join(data_dir,'training_samples.gpkg')
points = gpd.read_file(fn_points)
points

,lucas,survey_date,lc_class,tile_id,confidence,geometry
0,True,2015-06-19,311,9529,100.0,POINT (4770000.000 2404000.000)
1,False,2018-06-30,311,9529,85.0,POINT (4770132.229 2408502.635)
2,False,2000-06-30,324,9529,85.0,POINT (4770204.116 2414683.260)
3,False,2000-06-30,321,9529,85.0,POINT (4770502.478 2420085.850)
4,False,2006-06-30,321,9529,85.0,POINT (4770502.478 2420085.850)
...,...,...,...,...,...,...
754,False,2018-06-30,231,9529,85.0,POINT (4798660.699 2400143.955)
755,False,2006-06-30,311,9529,85.0,POINT (4798875.239 2408956.685)
756,False,2000-06-30,324,9529,85.0,POINT (4799014.807 2407048.185)
757,False,2000-06-30,311,9529,85.0,POINT (4799630.032 2407142.200)


... some **timeless** raster layers 

In [4]:
dir_timeless_layers = os.path.join(data_dir, 'images', 'timeless')
fn_timeless_layers = list(Path(dir_timeless_layers).glob('**/*.tif'))

print(f'Number of timeless layers: {len(fn_timeless_layers)}')

Number of timeless layers: 2


... and several **temporal** layers.

In [5]:
dir_temporal_layers = os.path.join(data_dir,'images')
fn_temporal_layers = list(Path(dir_temporal_layers).glob('????/*.tif'))

print(f'{len(fn_temporal_layers)} temporal layers from 2000 to 2020')

1743 temporal layers from 2000 to 2020


The association between the points and the temporal layers will occurs using the **survey_date** column

In [6]:
col_date = 'survey_date'

print('Number of samples per year:')
pd.to_datetime(points[col_date]).dt.year.value_counts()

Number of samples per year:


2018    207
2012    180
2006    180
2000    180
2015     12
Name: survey_date, dtype: int64

... and the name of **temporal** directories.

In [7]:
dirs = list(Path(dir_temporal_layers).glob('????'))
dirs.sort()

print('Temporal directories:')
for dir in dirs:
    n_layers = len(list(Path(os.path.join(dir_temporal_layers,dir)).glob('*.tif')))
    print(f' - {dir.name}: {n_layers} layers')

Temporal directories:
 - 2000: 85 layers
 - 2001: 85 layers
 - 2002: 85 layers
 - 2003: 85 layers
 - 2004: 85 layers
 - 2005: 85 layers
 - 2006: 85 layers
 - 2007: 85 layers
 - 2008: 85 layers
 - 2009: 85 layers
 - 2010: 85 layers
 - 2011: 85 layers
 - 2012: 85 layers
 - 2013: 85 layers
 - 2014: 85 layers
 - 2015: 85 layers
 - 2016: 85 layers
 - 2017: 85 layers
 - 2018: 85 layers
 - 2019: 85 layers
 - 2020: 43 layers


## Space Overlay

The points should be overlayed on all timeless layers, regardless the date information stored in survey_date column. In this case, we will use the **SpaceOverlay** class passing the arguments:
- *fn_points*: the geopackage filepath
- *dir_timeless_layers*: the directory containing the timeless raster files

In [8]:
spc_overlay = SpaceOverlay(fn_points, dir_timeless_layers, verbose=False)
timeless_data = spc_overlay.run()

Now we have the elevation and slope information for each points:

In [9]:
timeless_data

,lucas,survey_date,lc_class,tile_id,confidence,geometry,overlay_id,dtm_elevation,dtm_slope
0,True,2015-06-19,311,9529,100.0,POINT (4770000.000 2404000.000),1,721.0,3.952847
1,False,2018-06-30,311,9529,85.0,POINT (4770132.229 2408502.635),2,668.0,42.463219
2,False,2000-06-30,324,9529,85.0,POINT (4770204.116 2414683.260),3,720.0,12.261616
3,False,2000-06-30,321,9529,85.0,POINT (4770502.478 2420085.850),4,1121.0,3.726780
4,False,2006-06-30,321,9529,85.0,POINT (4770502.478 2420085.850),5,1121.0,3.726780
...,...,...,...,...,...,...,...,...,...
754,False,2018-06-30,231,9529,85.0,POINT (4798660.699 2400143.955),755,709.0,20.924799
755,False,2006-06-30,311,9529,85.0,POINT (4798875.239 2408956.685),756,698.0,12.923386
756,False,2000-06-30,324,9529,85.0,POINT (4799014.807 2407048.185),757,698.0,15.297966
757,False,2000-06-30,311,9529,85.0,POINT (4799630.032 2407142.200),758,718.0,46.785564


## Space-Time Overlay

For the temporal layers, the points should be filtered by year and overlayed on the right raster files. The **SpaceTimeOverlay** class implements this approach using the parameter:
* *timeless_data*: The result of SpaceOverlay (GeoPandas DataFrame) 
* *col_date*: The column that contains the date information (2018-09-13)
* *dir_temporal_layers*: The directory where the temporal raster files are stored, organized by year

In [10]:
spc_time_Overlay = SpaceTimeOverlay(timeless_data, col_date, dir_temporal_layers, verbose=False)
overlayed_data = spc_time_Overlay.run()

Now we have the elevation, slope, landsat and the night light data for each points:

In [11]:
overlayed_data

,lucas,survey_date,lc_class,tile_id,confidence,geometry,overlay_id,dtm_elevation,dtm_slope,landsat_ard_winter_green_p25,...,landsat_ard_fall_thermal_p25,landsat_ard_winter_thermal_p25,landsat_ard_spring_thermal_p75,landsat_ard_winter_swir2_p50,landsat_ard_spring_thermal_p50,landsat_ard_summer_thermal_p25,landsat_ard_summer_thermal_p50,night_lights,landsat_ard_summer_thermal_p75,landsat_ard_winter_thermal_p75
0,True,2015-06-19,311,9529,100.0,POINT (4770000.000 2404000.000),1,721.0,3.952847,7.0,...,187.0,185.0,188.0,30.0,186.0,187.0,187.0,0.180370,188.0,186.0
143,True,2015-06-19,321,9529,100.0,POINT (4772000.000 2406000.000),2,630.0,0.000000,3.0,...,187.0,187.0,189.0,12.0,188.0,188.0,189.0,0.000390,190.0,187.0
144,True,2015-06-26,322,9529,100.0,POINT (4772000.000 2424000.000),3,684.0,5.803495,4.0,...,186.0,186.0,186.0,17.0,185.0,186.0,187.0,0.615905,189.0,186.0
145,True,2015-06-19,311,9529,100.0,POINT (4772000.000 2402000.000),4,648.0,10.307764,2.0,...,187.0,188.0,186.0,18.0,185.0,187.0,187.0,0.293385,187.0,188.0
169,True,2015-10-20,322,9529,100.0,POINT (4788000.000 2422000.000),5,438.0,9.718253,15.0,...,186.0,187.0,188.0,36.0,186.0,185.0,185.0,0.535678,185.0,189.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734,False,2012-06-30,231,9529,85.0,POINT (4796422.302 2416657.855),176,666.0,25.873625,11.0,...,186.0,186.0,187.0,16.0,186.0,185.0,185.0,0.059816,185.0,186.0
736,False,2012-06-30,231,9529,85.0,POINT (4797303.915 2404796.255),177,702.0,6.718548,179.0,...,187.0,180.0,190.0,4.0,189.0,187.0,188.0,0.039074,188.0,180.0
738,False,2012-06-30,231,9529,85.0,POINT (4797638.685 2415216.855),178,560.0,6.508542,92.0,...,187.0,182.0,188.0,14.0,187.0,186.0,186.0,0.051485,187.0,182.0
742,False,2012-06-30,324,9529,85.0,POINT (4798052.494 2405145.480),179,658.0,17.159384,70.0,...,186.0,183.0,189.0,17.0,188.0,186.0,186.0,0.025304,186.0,183.0


## Save to CSV and GeoPackage files

At last, we need to save the overlayed points to access it in other softwares (QGIS) and eumap tutorial:

In [14]:
csv_output = os.path.join(data_dir, 'training_samples_overlayed.csv.gz')

print(f"Saving {csv_output}")
overlayed_data.to_csv(csv_output, compression='gzip')

Saving /home/leandro/Tmp/pilot_tiles/croatia_9529/training_samples_overlayed.csv.gz


In [13]:
gpkg_output =  os.path.join(data_dir, 'training_samples_overlayed.gpkg')

print(f"Saving {gpkg_output}")
overlayed_data.to_file(gpkg_output,  driver="GPKG")

Saving /home/leandro/Tmp/pilot_tiles/croatia_9529/training_samples_overlayed.gpkg
